In [1]:
from _shared import *

## Задача 1. Сколько выбросов удалять

С одной стороны, при удалении выбросов снижается дисперсия, что приводит к увеличению чувствительности теста. С другой стороны, при удалении выбросов уменьшается размер выборки, что приводит к уменьшению чувствительности.

Сравните мощности тестов с разной долей удаляемых данных:
- Используйте данные о времени работы бэкенда `2022-04-01T12_df_web_logs.csv` в период с __2022-03-01__ по __2022-03-08__.
- Уровень значимости — 0.05.
- Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений).
- Проверяем гипотезу о равенстве средних с помощью теста Стьюдента.
- Ожидаемый эффект — увеличение времени обработки на 1%.
- Эффект в синтетических А/В-тестах добавляем умножением на константу.

В ответ введите номера вариантов, упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между `np.quantile(values, 0.01)` и `np.quantile(values, 0.99)`. Квантили вычислять для каждой групп отдельно.